In [1]:
import openai
import notion_client
import json

with open('openai_secret_key') as r:
    openai_secret_key = r.readline().strip()
    openai.api_key = openai_secret_key

with open('notion_secret_key') as r:
    notion_secret_key = r.readline().strip()
    notion = notion_client.Client(auth=notion_secret_key)

# Entity extraction

In [4]:
engine = 'text-davinci-003'
journal_id = '66a8cd52520543df9895c373627f821d'
organizations_id = 'ec516282ca5248c3bd52d7cede238bc6'
people_id = '748131dafe774e4a9ad5c0c259f69aba'
places_id = '9c8884e3303a49238e44d9aad9d7e50e'
allowed_block_types = [
    'paragraph',
    'heading_1',
    'heading_2',
    'heading_3',
    'bulleted_list_item',
    'numbered_list_item',
    'to_do',
    'toggle',
    'quote'
]

In [47]:
entries = notion.databases.query(database_id=journal_id)

In [179]:
entry_id = 'c6affbed81284a09affcd41c8fa4915b'
entry = notion.pages.retrieve(page_id=entry_id)

In [5]:
def get_entry_plain_text(entry_id, allowed_block_types=allowed_block_types):
    blocks = notion.blocks.children.list(block_id=entry_id)['results']
    plain_text = ''

    for block in blocks:
        block_type = block['type']
        if block_type in allowed_block_types:
            rich_text = block[block_type]['rich_text']
            for text in rich_text:
                plain_text += text['plain_text']
            plain_text += '  \n\n'
    return plain_text

In [382]:
def get_entity_tags(entry_plain_text, engine='text-davinci-003'):
    extraction_prompt_template = """
    INSTRUCTIONS:
    From the text below, extract the following entities:
    Organizations: All platforms, products, companies and institutions mentioned by name (e.g. "Google", "FIFA", "World Bank"). Exclude broad terms such as industries ("Record Labels") that don't refer to specific companies. Exclude names of places (e.g. "Switzerland") and people (e.g. "Bill Gates").
    People: All people mentioned by their full name (e.g. John Doe). Exclude people mentioned only by first name (e.g. Peter).
    Places: All cities (e.g. Barcelona), regions (e.g. California) and countries (e.g. Italy) mentioned by name.
    Tags: At least 20 and ideally 50 of the most relevant, unusual, surprising and salient topics, ideas and themes the author is exploring and writing about. Also include topics that are mentioned in passing, or in relatively short fragments of the text. Make the topic names short and concise (e.g. use "Streaming Economics" instead of "The economics of music streaming")
    Capitalize each of the entities in title case.
    Format the results as a JSON dictionary containing all of the lists described above:
    {
        "organizations": [
            # list of organization entities goes here
        ],
        "people": [
            # list of people entities goes here
        ],
        "places": [
            # list of place entities goes here
        ],
        "tags": [
            # list of tag entities goes here
        ]
    }
    
    EXAMPLE TEXT:
    ###
    Pedro Cano and I were talking about the electric car industry and he suggested that we rent a Tesla for our trip to LA with Alex and Enric. I’m finding it harder to hate things lately. I find it hard to sympathize with people who immediately assign malice to people or companies they know little about, instead of trying to understand what they might not know. Despite not being a huge fan of Elon Musk, I don’t feel the need to join the online polarization about him and transfer that energy to my impressions of a Tesla.
    ###
    
    EXAMPLE OUTPUT:
    ###
    {
        "organizations": [
            "Tesla"
        ],
        "people": [
            "Pedro Cano", "Elon Musk"
        ],
        "places": [
            "Los Angeles"
        ],
        "tags": [
            "Electric Cars", "Empathy", "Hating", "Polarization", "Celebrities", "Online Behavior"
        ]
    }
    ###
    """
    

## Old stuff

In [278]:
extraction_prompt_template = """
From the text below, extract the following entities:
Organizations: All platforms, products, companies and institutions mentioned by name. Exclude broad terms such as industries ("Record Labels") that don't refer to specific companies. Exclude names of places and people.
People: All people mentioned by their full name (e.g. John Doe). Exclude people mentioned only by first name (e.g. Peter).
Places: All cities (e.g. Barcelona), regions (e.g. California) and countries (e.g. Italy) mentioned by name.
Tags: At least 20 and ideally 50 of the most relevant, unusual, surprising and salient topics, ideas and themes the author is exploring and writing about. Also include topics that are mentioned in passing, or in relatively short fragments of the text. Make the topic names short and concise (e.g. use "Streaming Economics" instead of "The economics of music streaming")
Format the results as a JSON dictionary containing all of the lists described above:
{
    "organizations": [
        # list of organization entities goes here
    ],
    "people": [
        # list of people entities goes here
    ],
    "places": [
        # list of place entities goes here
    ],
    "tags": [
        # list of tag entities goes here
    ]
}
"""
extraction_prompt_template += f"""
Text:###
{plain_text}
###
"""

In [279]:
extraction = openai.Completion.create(
    engine=engine,
    prompt=extraction_prompt,
    max_tokens=1024,
    temperature=0.2
)
entities_text = extraction['choices'][0]['text']

In [286]:
entities = json.loads(entities_text)

In [247]:
single_list_prompt = """
INSTRUCTIONS:
From the text below, make a list of all entities mentioned. 
Include all platforms, products, companies and institutions mentioned by name. 
Include all people mentioned. 
Include all places mentioned, including cities (e.g. Barcelona), regions (e.g. California) and countries (e.g. Italy) mentioned by name.
Include all topics, ideas and themes the author is writing about. 
Include topics that are mentioned in passing, or in relatively short fragments of the text.
Make the list long (at least 30 entries and 50 if possible) and exhaustive (include any entities regardless of how often or thoroughly they are mentioned).
Format the results as a JSON dictionary containing a single list of all entities extracted as per above:
{
    "entities": [
        # list of all extracted entities goes here
    ]
}
"""
single_list_prompt += f"""
TEXT:###
{plain_text}
###
"""

In [248]:
single_list_results = openai.Completion.create(
    engine=engine,
    prompt=single_list_prompt,
    max_tokens=1500,
    temperature=0.9
)
single_list = single_list_results['choices'][0]['text']

In [249]:
print(single_list)

{
    "entities": [
        "Dominican barber",
        "Nic",
        "Spanish",
        "American",
        "Canarian",
        "Catalan",
        "Spain",
        "Catalonia",
        "United States",
        "Canary Islands",
        "Business",
        "Philosophy of Life",
        "Brian Schmitt",
        "Fort Greene",
        "Prospect Park",
        "Design",
        "Apple",
        "Product Strategy and Mangement",
        "BMAT",
        "Carina",
        "Songtradr",
        "Design Company",
        "Live Experience of Music",
        "Underdogs Function",
        "Asymmetric Bets",
        "Haircut",
        "Racial Stereotyping",
        "Mexicans",
        "LA",
        "Black People",
        "Minz",
        "Korean People",
        "Asian People",
        "Julie",
        "Japanese Mom",
        "Racially Homogeneous Cultures",
        "Island Nations",
        "United Kingdom",
        "Ireland",
        "Japan",
        "Mixed Cultures",
        "Mixed Races",
    

In [281]:
filtering_prompt = """
Below are four lists of entities: People, Organizations, Places and Tags.
Filter each of the lists using the following instructions:
- People: keep only entries in the input list that are people referred to exclusively by their full name (e.g. 'John Doe'). Exclude from this list people referred to by their first name only (e.g. 'John').
- Organizations: keep only entries in the input list that refer to specific platforms (e.g. 'Twitter'), companies (e.g. 'Apple'), and institutions (e.g. 'NATO'). Do not include names of industries (e.g. 'Music Publishing') or specific products (e.g. 'iPhone').
- Places: keep only entries in the input list that refer to specific neighborhoods (e.g. 'Eixample'), cities (e.g. 'Barcelona'), regions (e.g. 'Catalonia'), countries (e.g. 'Spain'), continents (e.g. 'Asia') or natural geographical landmarks (e.g. 'The Himalayas').
- Tags: keep only entries in the input list that aren't already included in the lists above.
Capitalize all entries in title case.
Format the results as a JSON dictionary containing all of the lists described above:
{
    "organizations": [
        # list of organization entities goes here
    ],
    "people": [
        # list of people entities goes here
    ],
    "places": [
        # list of place entities goes here
    ],
    "tags": [
        # list of tag entities goes here
    ]
}
"""

filtering_prompt += f"""
List:###
{entities_text}
###
"""

In [282]:
filtering = openai.Completion.create(
    engine=engine,
    prompt=filtering_prompt,
    max_tokens=1024,
    temperature=0.3
)
filtered_text = filtering['choices'][0]['text']

In [284]:
print(filtered_text)


Filtered List:

{
    "organizations": [
        "Apple",
        "BMAT",
        "Songtradr"
    ],
    "people": [
        "Brian Schmitt",
        "Carina",
        "Minz",
        "Julie",
        "Mel",
        "JJ",
        "Zal"
    ],
    "places": [
        "Dominican Republic",
        "Spain",
        "Catalonia",
        "United States",
        "Canary Islands",
        "The Philippines",
        "Scandinavia",
        "South Korea",
        "Japan"
    ],
    "tags": [
        "Cultures of Me",
        "Underdog Functions",
        "Rabbit Hole",
        "Stereotypes",
        "Beauty",
        "Island Nations",
        "K-Pop",
        "Fandom",
        "World of Warcraft",
        "Metal Music",
        "Tumblr",
        "Emo Music",
        "Middle-Class Upbringing",
        "Financial Struggle",
        "Psychological Struggle",
        "Social Justice",
        "Atheistic Liberals",
        "Journaling",
        "Narratives",
        "Mood Swings",
        "Cycles",

# Entity retrieval/creation and relation creation

In [6]:
entry_id = '7ee6ee7d5ecf47679848f0ef0954bfd1'
entry = notion.pages.retrieve(page_id=entry_id)
entry_plain_text = get_entry_plain_text(entry_id)
#print(entry_plain_text)

In [7]:
entry

{'object': 'page',
 'id': '7ee6ee7d-5ecf-4767-9848-f0ef0954bfd1',
 'created_time': '2022-12-29T16:28:00.000Z',
 'last_edited_time': '2022-12-30T22:41:00.000Z',
 'created_by': {'object': 'user',
  'id': 'fef31225-60a6-48ca-9c0a-78051140f2ff'},
 'last_edited_by': {'object': 'user',
  'id': '42bea9d8-261e-416e-a0cd-8e90482f044d'},
 'cover': None,
 'icon': {'type': 'external',
  'external': {'url': 'https://www.notion.so/icons/book-closed_gray.svg'}},
 'parent': {'type': 'database_id',
  'database_id': '66a8cd52-5205-43df-9895-c373627f821d'},
 'archived': False,
 'properties': {'Dots': {'id': '%3EBvH',
   'type': 'relation',
   'relation': [],
   'has_more': False},
  'Media': {'id': 'Ajzp',
   'type': 'relation',
   'relation': [],
   'has_more': False},
  'Date': {'id': 'SJ%3AQ',
   'type': 'date',
   'date': {'start': '2022-12-29', 'end': None, 'time_zone': None}},
  'Places': {'id': 'VGoF',
   'type': 'relation',
   'relation': [{'id': '181679fb-61a3-4ddc-9806-70e3c3eae0c2'}],
   'has_

In [27]:
journal_id = '66a8cd52520543df9895c373627f821d'
organizations_id = 'ec516282ca5248c3bd52d7cede238bc6'
people_id = '748131dafe774e4a9ad5c0c259f69aba'
places_id = '9c8884e3303a49238e44d9aad9d7e50e'
tags_id = 'e01ea0835efe472e82bf197f9a16a92f'

entity_types = {
    'organizations': {
        'database_id': organizations_id,
        'response_key': 'organizations',
        'property_name': 'Organizations',
        'title_property': 'Name',
        'emoji': '🏢'
    },
    'people': {
        'database_id': people_id,
        'response_key': 'people',
        'property_name': 'People',
        'title_property': 'Name',
        'emoji': '👤'
    },
    'places': {
        'database_id': places_id,
        'response_key': 'places',
        'property_name': 'Places',
        'title_property': 'Name',
        'emoji': '📌'
    },
    'tags': {
        'database_id': tags_id,
        'response_key': 'tags',
        'property_name': 'Tags',
        'title_property': 'Name',
        'emoji': '🏷'
    },
    'journal': {
        'database_id': journal_id,
        'response_key': '',
        'property_name': '',
        'title_property': 'Name',
        'emoji': ''
    }
}

In [383]:
entity_tags = get_entity_tags(entry_plain_text)
entity_tags

{'organizations': ['BMAT', 'Songtradr', 'Apple'],
 'people': ['Nic', 'Brian Schmitt', 'Carina', 'Minz', 'Julie', 'Mel'],
 'places': ['Fort Greene',
  'Prospect Park',
  'Los Angeles',
  'The Philippines',
  'Scandinavia',
  'South Korea',
  'United Kingdom',
  'Ireland',
  'Japan'],
 'tags': ['Racial Stereotyping',
  'Racism',
  'Genetic Diversity',
  'Beauty Culture',
  'Fandom',
  'Escapism',
  'Middle-Class Upbringing',
  'Financial Struggle',
  'Psychological Struggle',
  'Social Justice',
  'Atheistic Liberals',
  'Mood Swings',
  'American Worldview']}

In [459]:
def get_all_pages(database_id):
    pages = []
    response = notion.databases.query(database_id=database_id)
    pages += response['results']
    while response['has_more']:
        response = notion.databases.query(database_id, start_cursor=response['next_cursor'])
        pages += response['results']
    return pages


def get_pages_dict(raw_pages, title_property='Name'):
    # Parse all pages into a dict {'page_title': page}
    pages = {}
    for page in raw_pages:
        try:
            name = page['properties'][title_property]['title'][0]['plain_text']
        except:
            pass
        pages[name] = page
    return pages

def get_new_property_dict(title, title_property='Name'):
    property_dict = {
        title_property: {
            'title': [
                {
                    'text': {
                        'content': title
                    }
                }
            ]
        }
    }
    return property_dict

In [445]:
raw_pages = get_all_pages(database_id)
pages_dict = get_pages_dict(raw_pages)

In [358]:
# take original entry
# for each autotagged entity type
#  get all existing pages of that type
#  for each autotagged entity of that type
#   if it doesn't have a page, create it
#   add a relation to the page in the original entry

In [441]:
title_property = 'Name'
relation_ids = []
database_id = places_id
database_emoji = "📌"

for tag in entity_tags['places']:
    page = pages_dict.get(tag)
    if page:
        relation_ids.append({'id': page['id']})
    else:
        property_dict = get_new_property_dict(tag, title_property)
        parent_dict = {'database_id': database_id}
        icon_dict = {'type': 'emoji', 'emoji': database_emoji}
        new_page = notion.pages.create(parent=parent_dict, properties=property_dict, icon=icon_dict)
        relation_ids.append({'id': new_page['id']})

prop = entry['properties']['Places']
prop['relation'] = prop['relation'] + relation_ids
property_updates = {'Places': prop}
page = notion.pages.update(page_id=entry_id, properties=property_updates)

In [462]:
def add_relations(entry, entity_tags, entity_types):
    for entity_type, entity_data in entity_types.items():
        database_id = entity_data['database_id']
        response_key = entity_data['response_key']
        property_name = entity_data['property_name']
        emoji = entity_data['emoji']
        
        raw_pages = get_all_pages(database_id)
        pages_dict = get_pages_dict(raw_pages)
        relation_ids = []
        for tag in entity_tags[response_key]:
            page = pages_dict.get(tag)
            if page:
                relation_ids.append({'id': page['id']})
            else:
                property_dict = get_new_property_dict(tag, title_property)
                parent_dict = {'database_id': database_id}
                icon_dict = {'type': 'emoji', 'emoji': emoji}
                new_page = notion.pages.create(parent=parent_dict, properties=property_dict, icon=icon_dict)
                relation_ids.append({'id': new_page['id']})
        prop = entry['properties'][property_name]
        prop['relation'] = prop['relation'] + relation_ids
        property_updates = {property_name: prop}
        page = notion.pages.update(page_id=entry_id, properties=property_updates)

In [461]:
add_relations(entry, entity_tags, entity_types)

{'Organizations': {'id': 'onrg', 'type': 'relation', 'relation': [{'id': 'c5f3cf7e-0ac5-443f-bac9-7c0b24dfab58'}, {'id': '58332b06-4c0e-4ce3-8fa3-acc7ecc5d5c5'}, {'id': '492da4fc-e3b2-4669-bf9d-c913c08de725'}, {'id': 'c5f3cf7e-0ac5-443f-bac9-7c0b24dfab58'}, {'id': '58332b06-4c0e-4ce3-8fa3-acc7ecc5d5c5'}, {'id': '492da4fc-e3b2-4669-bf9d-c913c08de725'}], 'has_more': False}}
{'People': {'id': 'bgYk', 'type': 'relation', 'relation': [{'id': '16b09804-367e-4b8f-82f6-9cf49e88b71f'}, {'id': 'a11eb426-35a9-4085-ad37-2a0dc63b81e9'}, {'id': '16b09804-367e-4b8f-82f6-9cf49e88b71f'}, {'id': 'eb9fd646-754b-43d9-b676-448a5f7aca70'}, {'id': '9d2cfb21-8c29-40b3-be73-968355c4ee98'}, {'id': 'ca293ea1-48a7-4434-85a0-579c4972309b'}, {'id': '5c5822da-a908-4065-8d8f-a63456c474a7'}, {'id': '247d751c-6f27-4099-8d84-9398bc1cd677'}, {'id': '16b09804-367e-4b8f-82f6-9cf49e88b71f'}, {'id': 'daf73205-809e-44df-84ff-1f011af9f940'}, {'id': 'a7ed2885-e540-4f5d-a47c-e3a6c7f2ed87'}, {'id': 'f41fc9fc-e15f-46cb-bcad-ecc85e

In [457]:
entity_tags

{'organizations': ['BMAT', 'Songtradr', 'Apple'],
 'people': ['Nic', 'Brian Schmitt', 'Carina', 'Minz', 'Julie', 'Mel'],
 'places': ['Fort Greene',
  'Prospect Park',
  'Los Angeles',
  'The Philippines',
  'Scandinavia',
  'South Korea',
  'United Kingdom',
  'Ireland',
  'Japan'],
 'tags': ['Racial Stereotyping',
  'Racism',
  'Genetic Diversity',
  'Beauty Culture',
  'Fandom',
  'Escapism',
  'Middle-Class Upbringing',
  'Financial Struggle',
  'Psychological Struggle',
  'Social Justice',
  'Atheistic Liberals',
  'Mood Swings',
  'American Worldview']}

In [475]:
def auto_tag(entry_id, entity_types):
    entry = notion.pages.retrieve(page_id=entry_id)
    entry_plain_text = get_entry_plain_text(entry_id)
    entity_tags = get_entity_tags(entry_plain_text)
    add_relations(entry, entity_tags, entity_types)

In [477]:
auto_tag('7ee6ee7d5ecf47679848f0ef0954bfd1', entity_types)

In [478]:
page = notion.pages.retrieve(page_id='7ee6ee7d5ecf47679848f0ef0954bfd1')

In [479]:
page

{'object': 'page',
 'id': '7ee6ee7d-5ecf-4767-9848-f0ef0954bfd1',
 'created_time': '2022-12-29T16:28:00.000Z',
 'last_edited_time': '2022-12-30T22:41:00.000Z',
 'created_by': {'object': 'user',
  'id': 'fef31225-60a6-48ca-9c0a-78051140f2ff'},
 'last_edited_by': {'object': 'user',
  'id': '42bea9d8-261e-416e-a0cd-8e90482f044d'},
 'cover': None,
 'icon': {'type': 'external',
  'external': {'url': 'https://www.notion.so/icons/book-closed_gray.svg'}},
 'parent': {'type': 'database_id',
  'database_id': '66a8cd52-5205-43df-9895-c373627f821d'},
 'archived': False,
 'properties': {'Dots': {'id': '%3EBvH',
   'type': 'relation',
   'relation': [],
   'has_more': False},
  'Media': {'id': 'Ajzp',
   'type': 'relation',
   'relation': [],
   'has_more': False},
  'Date': {'id': 'SJ%3AQ',
   'type': 'date',
   'date': {'start': '2022-12-29', 'end': None, 'time_zone': None}},
  'Places': {'id': 'VGoF',
   'type': 'relation',
   'relation': [{'id': '181679fb-61a3-4ddc-9806-70e3c3eae0c2'}],
   'has_

In [8]:
extraction_prompt_template = """
INSTRUCTIONS:
From the text below, extract the following entities:
Organizations: All platforms, products, companies and institutions mentioned by name (e.g. "Google", "FIFA", "World Bank"). Exclude broad terms such as industries ("Record Labels") that don't refer to specific companies. Exclude names of places (e.g. "Switzerland") and people (e.g. "Bill Gates").
People: All people mentioned by their full name (e.g. John Doe). Exclude people mentioned only by first name (e.g. Peter).
Places: All cities (e.g. Barcelona), regions (e.g. California) and countries (e.g. Italy) mentioned by name.
Tags: At least 20 and ideally 50 of the most relevant, unusual, surprising and salient topics, ideas and themes the author is exploring and writing about. Also include topics that are mentioned in passing, or in relatively short fragments of the text. Make the topic names short and concise (e.g. use "Streaming Economics" instead of "The economics of music streaming")
Capitalize each of the entities in title case.
Format the results as a JSON dictionary containing all of the lists described above:
{
    "organizations": [
        # list of organization entities goes here
    ],
    "people": [
        # list of people entities goes here
    ],
    "places": [
        # list of place entities goes here
    ],
    "tags": [
        # list of tag entities goes here
    ]
}

EXAMPLE TEXT:
###
Pedro Cano and I were talking about the electric car industry and he suggested that we rent a Tesla for our trip to LA with Alex and Enric. I’m finding it harder to hate things lately. I find it hard to sympathize with people who immediately assign malice to people or companies they know little about, instead of trying to understand what they might not know. Despite not being a huge fan of Elon Musk, I don’t feel the need to join the online polarization about him and transfer that energy to my impressions of a Tesla.
###

EXAMPLE OUTPUT:
###
{
    "organizations": [
        "Tesla"
    ],
    "people": [
        "Pedro Cano", "Elon Musk"
    ],
    "places": [
        "Los Angeles"
    ],
    "tags": [
        "Electric Cars", "Empathy", "Hating", "Polarization", "Celebrities", "Online Behavior"
    ]
}
###
"""

In [9]:
prompts = {
    'entity_extraction': extraction_prompt_template
}

In [30]:
with open('prompts.json', 'w') as f:
    json.dump(prompts, f, indent=4)

In [15]:
with open('entity_types.json') as f:
    prompts_dict = json.load(f)

In [18]:
print(prompts_dict['entity_extraction'])


INSTRUCTIONS:
From the text below, extract the following entities:
Organizations: All platforms, products, companies and institutions mentioned by name (e.g. "Google", "FIFA", "World Bank"). Exclude broad terms such as industries ("Record Labels") that don't refer to specific companies. Exclude names of places (e.g. "Switzerland") and people (e.g. "Bill Gates").
People: All people mentioned by their full name (e.g. John Doe). Exclude people mentioned only by first name (e.g. Peter).
Places: All cities (e.g. Barcelona), regions (e.g. California) and countries (e.g. Italy) mentioned by name.
Tags: At least 20 and ideally 50 of the most relevant, unusual, surprising and salient topics, ideas and themes the author is exploring and writing about. Also include topics that are mentioned in passing, or in relatively short fragments of the text. Make the topic names short and concise (e.g. use "Streaming Economics" instead of "The economics of music streaming")
Capitalize each of the entities 

In [20]:
page=entry

In [25]:
page['properties']['Name']['title'][0]['plain_text']

'December 29th 2022'

In [26]:
page

{'object': 'page',
 'id': '7ee6ee7d-5ecf-4767-9848-f0ef0954bfd1',
 'created_time': '2022-12-29T16:28:00.000Z',
 'last_edited_time': '2022-12-30T22:41:00.000Z',
 'created_by': {'object': 'user',
  'id': 'fef31225-60a6-48ca-9c0a-78051140f2ff'},
 'last_edited_by': {'object': 'user',
  'id': '42bea9d8-261e-416e-a0cd-8e90482f044d'},
 'cover': None,
 'icon': {'type': 'external',
  'external': {'url': 'https://www.notion.so/icons/book-closed_gray.svg'}},
 'parent': {'type': 'database_id',
  'database_id': '66a8cd52-5205-43df-9895-c373627f821d'},
 'archived': False,
 'properties': {'Dots': {'id': '%3EBvH',
   'type': 'relation',
   'relation': [],
   'has_more': False},
  'Media': {'id': 'Ajzp',
   'type': 'relation',
   'relation': [],
   'has_more': False},
  'Date': {'id': 'SJ%3AQ',
   'type': 'date',
   'date': {'start': '2022-12-29', 'end': None, 'time_zone': None}},
  'Places': {'id': 'VGoF',
   'type': 'relation',
   'relation': [{'id': '181679fb-61a3-4ddc-9806-70e3c3eae0c2'}],
   'has_